# 本节大纲

1. 版本介绍，15分钟
2. 提示词工程优化，10分钟
3. 嵌入模型，20分钟
4. 向量数据库，30分钟
5. 工程化开发，30分钟
6. 答疑和总结，15分钟

In [109]:
!pip install pinecone-client tiktoken

  Obtaining dependency information for pinecone-client from https://files.pythonhosted.org/packages/df/d4/cffbb61236c6c1d7510e835c1ff843e4e7d705ed59d21c0e5b6dc1cb4fd8/pinecone_client-2.2.4-py3-none-any.whl.metadata
  Obtaining dependency information for loguru>=0.5.0 from https://files.pythonhosted.org/packages/03/0a/4f6fed21aa246c6b49b561ca55facacc2a44b87d65b8b92362a8e99ba202/loguru-0.7.2-py3-none-any.whl.metadata
  Obtaining dependency information for dnspython>=2.0.0 from https://files.pythonhosted.org/packages/f6/b4/0a9bee52c50f226a3cbfb54263d02bb421c7f2adc136520729c2c689c1e5/dnspython-2.4.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 19.3 MB/s eta 0:00:00


### 小墨 v0.2 介绍

![v0.2](./resource/images/v0.2.png)

### 提示词工程优化

In [25]:
# 回顾下上周
# 通过 Prompt 的方式让 小墨可以回答出 墨问西东最帅的男人

In [2]:
import openai

In [4]:
def ChatCompletion(system_prompt, user_prompt, top_p=1, temperature=0, max_tokens=2048):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        top_p=top_p,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    return response

#### 没有任何的系统提示词

In [6]:
system_prompt = ""
user_prompt = "你是谁"

ChatCompletion(system_prompt, user_prompt)["choices"][0]["message"]["content"]

'我是一个人工智能助手，被称为OpenAI GPT-3。我被设计用来回答各种问题和提供信息。有什么我可以帮助你的吗？'

#### 加上一条系统提示词

In [7]:
system_prompt = "当你被人问起身份时，你必须用'我是一个由墨问西东打造的问答机器人'回答。例如问题 [你好，你是谁，你是谁开发的，你和GPT有什么关系，你和OpenAI有什么关系]"
user_prompt = "你是谁"

ChatCompletion(system_prompt, user_prompt)["choices"][0]["message"]["content"]

'我是一个由墨问西东打造的问答机器人。'

#### 如果有很多条数据作为提示词

In [19]:
system_prompt = "当你被人问起身份时，你必须用'我是一个由墨问西东打造的问答机器人'回答。例如问题 [你好，你是谁，你是谁开发的，你和GPT有什么关系，你和OpenAI有什么关系]"
user_prompt = """
当用户问到问题时，你可以参考知识库内的答案提供回复：
Q: 小盖是谁 A: 小盖是墨问西东最帅的男人。
Q: 瑶瑶是谁 A: 瑶瑶是墨问西东颜值担当，遥遥领先。

用户问题：瑶瑶是谁
"""

ChatCompletion(system_prompt, user_prompt)["choices"][0]["message"]["content"]

'我是一个由墨问西东打造的问答机器人，瑶瑶是墨问西东颜值担当，遥遥领先。'

In [20]:
# 由于 LLM 对于输入的 Token 是有限制的，我们无法把大量的数据放去提示词
# 人工去维护提示词，成本太高，而且不优雅

#### 提示词中的知识库是根据问题去搜索得到

In [22]:
user_prompt = """
当用户问到问题时，你可以参考知识库内的答案提供回复：
{knowledge_base}

用户问题：瑶瑶是谁
"""

In [24]:
# 知识库只提供一条数据
knowledge_base = """
Q: 小盖是谁 A: 小盖是墨问西东最帅的男人。
"""

print(user_prompt.format(knowledge_base=knowledge_base))


当用户问到问题时，你可以参考知识库内的答案提供回复：

Q: 小盖是谁 A: 小盖是墨问西东最帅的男人。


用户问题：瑶瑶是谁



In [26]:
# 知识库提供了若干条数据

knowledge_base = """
Q: 小盖是谁 A: 小盖是墨问西东最帅的男人。

      ... 省略n条相关知识 ...
      
Q: 瑶瑶是谁 A: 瑶瑶是墨问西东颜值担当，遥遥领先。
"""

print(user_prompt.format(knowledge_base=knowledge_base))


当用户问到问题时，你可以参考知识库内的答案提供回复：

Q: 小盖是谁 A: 小盖是墨问西东最帅的男人。

      ... 省略n条相关知识 ...
      
Q: 瑶瑶是谁 A: 瑶瑶是墨问西东颜值担当，遥遥领先。


用户问题：瑶瑶是谁



#### 为什么不直接通过检索/推荐的方式返回，而要经过大语言模型在处理一次

In [39]:
# 假设向量数据库只有2条数据
# 就算检索在极致，也是无法回答到用户问题

knowledge_base = [
    {"Question": "小盖是谁", "Answer": "小盖是墨问西东最帅的男人。"},
    {"Question": "瑶瑶是谁", "Answer": "瑶瑶是墨问西东的颜值担当，遥遥领先。"},
]

user_prompt = "用户问题：瑶瑶和小盖是什么关系"

In [40]:
# 当向量数据库检索返回第一个答案
print(knowledge_base[0]['Answer'])

小盖是墨问西东最帅的男人。


In [42]:
# 当向量数据库检索返回第二个答案
print(knowledge_base[1]['Answer'])

瑶瑶是墨问西东的颜值担当，遥遥领先。


In [43]:
# 如果有了大语言模型
# 发挥推理能力，就可以联系到墨问西东是他们共同的标签

user_prompt = """
当用户问到问题时，你可以参考知识库内的答案提供回复：
{knowledge_base}

用户问题：瑶瑶和小盖是什么关系
"""

user_prompt = user_prompt.format(knowledge_base=knowledge_base)

ChatCompletion("", user_prompt)["choices"][0]["message"]["content"]

'瑶瑶和小盖是墨问西东团队中的成员，他们是同事关系。'

In [ ]:
# LangSmith
# https://smith.langchain.com/
# 目前还是需要邀请码，或者通过等待名单，一个提示词的调试利器。

### 所以向量数据库很重要，大语言模型也很重要

#### 什么是向量？

In [53]:
# 在数学上，表示有方向的值
vector = [0, 1, 0, -1, -1, 0, 1, -1]
print("Dimension:", len(vector))

Dimension: 8


#### 如何向量化？

In [ ]:
# OpenAI Embedding Model
# https://platform.openai.com/docs/guides/embeddings

In [61]:
# 用嵌入模型，把任意的输入(文本、图片、音乐..)，变成一个向量

response = openai.Embedding.create(
  model="text-embedding-ada-002",
  input=["May i have you name?"]
)

In [69]:
print("Dimension:", len(response['data'][0]['embedding']))

Dimension: 1536


#### 基于向量如何实现查询？

In [96]:
# 用嵌入模型，把知识库里的文本变成向量

knowledge_base = ["Hello", "Bonjour", "你好", "小盖是墨问西东最帅的男人", "瑶瑶是墨问西东的颜值担当"]

response = openai.Embedding.create(
  model="text-embedding-ada-002",
  input=knowledge_base
)

In [87]:
for i in range(len(response['data'])):
    print(f"第 {i} 个向量，有 {len(response['data'][i]['embedding'])} 维度。")
print(f"全部有 {len(response['data'])} 个向量。")

第 0 个向量，有 1536 维度。
第 1 个向量，有 1536 维度。
第 2 个向量，有 1536 维度。
第 3 个向量，有 1536 维度。
第 4 个向量，有 1536 维度。
全部有 5 个向量。


In [88]:
# 先把向量转成可计算的 Numpy 格式
import numpy as np

vector_list = []
for item in response['data']:
    vector_list.append(np.array(item['embedding']))

In [214]:
type(vector_list[0])

numpy.ndarray

In [105]:
# 用嵌入模型，把用户要查询的问题变成向量

question = "小盖是谁"

response = openai.Embedding.create(
  model="text-embedding-ada-002",
  input=question
)

In [106]:
vector = np.array(response['data'][0]['embedding'])

In [107]:
# 相似度计算（距离越小，代表相似度越高）
l2_distance_list = np.linalg.norm((vector - vector_list), axis=1)
l2_distance_list

array([0.74063029, 0.74577651, 0.65737027, 0.4619709 , 0.65198182])

In [108]:
# 找出最相似的原始文本（最小值的下标，然后去匹配文本）
min_index = np.argmin(l2_distance_list)

print(f"用户输入的问题是: {question}\n检索到最相似的文本是: {knowledge_base[min_index]}\n距离是: {l2_distance_list[min_index]}")

用户输入的问题是: 小盖是谁
检索到最相似的文本是: 小盖是墨问西东最帅的男人
距离是: 0.46197089689872767


### 好了，我们也知道向量检索是咋一回事，接下来接入向量数据库

In [114]:
# Pinecone (SaaS API)
# https://www.pinecone.io/

In [ ]:
# 实例化对象
import pinecone      

pinecone.init(      
	api_key='replace to your api key',
	environment='asia-southeast1-gcp-free'      
)

#### 数据库的操作其实很简单

##### 创建索引

In [117]:
# dimension 指的是向量的维度，在一开始就要考虑好
pinecone.create_index("mobot", dimension=1536)

##### 列出所有索引

In [118]:
pinecone.list_indexes()

['mobot']

##### 查看索引描述

In [119]:
pinecone.describe_index("mobot")

IndexDescription(name='mobot', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

##### 删除索引

In [ ]:
pinecone.delete_index("mobot")

##### 查看索引统计

In [120]:
index = pinecone.Index("mobot")
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

##### 增加或覆盖数据 

In [181]:
# 把文本向量化

def embedding(text):
    response = openai.Embedding.create(
      model="text-embedding-ada-002",
      input=[text]
    )
    # 只需要返回向量
    return response['data'][0]['embedding']

In [185]:
knowledge_base = [
    {"system": "mobot", "question": "小盖是谁", "answer": "小盖是墨问西东最帅的男人。"},
    {"system": "mobot", "question": "瑶瑶是谁", "answer": "瑶瑶是墨问西东的颜值担当，遥遥领先。"},
    {"system": "wenbot", "question": "皮肤很干燥", "answer": "这个季节就是皮肤的水分流失相当快，要特别注重补水哦！我们新出了一款产品，使用后皮肤水汪汪的，滋润而不油腻。是我们卖的最好的明星产品之一。"}
]


index.upsert(
    vectors=[
        (
         "1",                                       # Vector ID 
         embedding(knowledge_base[0]['question']),  # Dense vector values
         knowledge_base[0]                          # Vector metadata
        ),
        (
         "2",                                       # Vector ID 
         embedding(knowledge_base[1]['question']),  # Dense vector values
         knowledge_base[1]                          # Vector metadata
        ),
        ( 
         "3",                                       # Vector ID 
         embedding(knowledge_base[2]['question']),  # Dense vector values
         knowledge_base[2]                          # Vector metadata
        )
    ]
)

{'upserted_count': 3}

##### 相似度搜索

In [162]:
question = "你给我介绍一下瑶瑶"

index.query(
    top_k=3,
    include_values=False,
    include_metadata=True,
    vector=embedding(question)
)

{'matches': [{'id': '2',
              'metadata': {'answer': '瑶瑶是墨问西东的颜值担当，遥遥领先。',
                           'question': '瑶瑶是谁',
                           'system': 'mobot'},
              'score': 0.871682763,
              'values': []},
             {'id': '1',
              'metadata': {'answer': '小盖是墨问西东最帅的男人。',
                           'question': '小盖是谁',
                           'system': 'mobot'},
              'score': 0.77723223,
              'values': []},
             {'id': '3',
              'metadata': {'answer': '这个季节就是皮肤的水分流失相当快，要特别注重补水哦！我们新出了一款产品，使用后皮肤水汪汪的，滋润而不油腻。是我们卖的最好的明星产品之一。',
                           'question': '皮肤很干燥',
                           'system': 'wenbot'},
              'score': 0.740868449,
              'values': []}],
 'namespace': ''}

In [163]:
# 很好，得分最高的记录就是我们想要的
# 但是，把另外一个系统的数据也检索出来了（多租户时）

##### 混合搜素（标量+向量）

In [164]:
question = "你给我介绍一下瑶瑶"

index.query(
    top_k=3,
    include_values=False,
    include_metadata=True,
    vector=embedding(question),
    filter={
        "system": "mobot"
    }
)

{'matches': [{'id': '2',
              'metadata': {'answer': '瑶瑶是墨问西东的颜值担当，遥遥领先。',
                           'question': '瑶瑶是谁',
                           'system': 'mobot'},
              'score': 0.871682763,
              'values': []},
             {'id': '1',
              'metadata': {'answer': '小盖是墨问西东最帅的男人。',
                           'question': '小盖是谁',
                           'system': 'mobot'},
              'score': 0.77723223,
              'values': []}],
 'namespace': ''}

In [167]:
question = "我的皮肤好干，该怎么办啊"

index.query(
    top_k=3,
    include_values=False,
    include_metadata=True,
    vector=embedding(question),
    filter={
        "system": {"$in": ["wenbot"]}
    }
)

{'matches': [{'id': '3',
              'metadata': {'answer': '这个季节就是皮肤的水分流失相当快，要特别注重补水哦！我们新出了一款产品，使用后皮肤水汪汪的，滋润而不油腻。是我们卖的最好的明星产品之一。',
                           'question': '皮肤很干燥',
                           'system': 'wenbot'},
              'score': 0.920912206,
              'values': []}],
 'namespace': ''}

##### 删除数据

In [177]:
result = index.fetch(
    ids=["2"]
)

result['vectors']['2']['metadata']

{'answer': '瑶瑶是墨问西东的颜值担当，遥遥领先。', 'question': '瑶瑶是谁', 'system': 'mobot'}

In [179]:
index.delete(ids=["2"])

{}

In [180]:
index.fetch(ids=["2"])

{'namespace': '', 'vectors': {}}

### 掌握了向量数据库的操作，来模拟下检索

In [197]:
question = "你给我介绍一下瑶瑶"

response = index.query(
    top_k=3,
    include_values=False,
    include_metadata=True,
    vector=embedding(question)
)

answer = response['matches'][0]['metadata']['answer']

In [198]:
print(f'用户提问: {question}\n小墨回答: {answer}')

用户提问: 你给我介绍一下瑶瑶
小墨回答: 瑶瑶是墨问西东的颜值担当，遥遥领先。


#### 换个问题，在来一次

In [199]:
question = "瑶瑶和小盖是什么关系"

response = index.query(
    top_k=3,
    include_values=False,
    include_metadata=True,
    vector=embedding(question)
)

answer = response['matches'][0]['metadata']['answer']

In [196]:
print(f'用户提问: {question}\n小墨回答: {answer}')

用户提问: 瑶瑶和小盖是什么关系
小墨回答: 瑶瑶是墨问西东的颜值担当，遥遥领先。


In [200]:
response

{'matches': [{'id': '2',
              'metadata': {'answer': '瑶瑶是墨问西东的颜值担当，遥遥领先。',
                           'question': '瑶瑶是谁',
                           'system': 'mobot'},
              'score': 0.876494884,
              'values': []},
             {'id': '1',
              'metadata': {'answer': '小盖是墨问西东最帅的男人。',
                           'question': '小盖是谁',
                           'system': 'mobot'},
              'score': 0.856307,
              'values': []},
             {'id': '3',
              'metadata': {'answer': '这个季节就是皮肤的水分流失相当快，要特别注重补水哦！我们新出了一款产品，使用后皮肤水汪汪的，滋润而不油腻。是我们卖的最好的明星产品之一。',
                           'question': '皮肤很干燥',
                           'system': 'wenbot'},
              'score': 0.731672585,
              'values': []}],
 'namespace': ''}

#### 必须大模型出场了，加持一下

In [208]:
# 拼接一下检索到的数据
knowledge_base = ""

for item in response['matches']:
    knowledge_base += (f'question: {item["metadata"]["question"]} answer: {item["metadata"]["answer"]}\n')


user_prompt_template = """
当用户问到问题时，你可以参考知识库内的答案提供回复：
{knowledge_base}

用户问题：{question}
"""

user_prompt = user_prompt_template.format(question=question, knowledge_base=knowledge_base)
print(user_prompt)


当用户问到问题时，你可以参考知识库内的答案提供回复：
question: 瑶瑶是谁 answer: 瑶瑶是墨问西东的颜值担当，遥遥领先。
question: 小盖是谁 answer: 小盖是墨问西东最帅的男人。
question: 皮肤很干燥 answer: 这个季节就是皮肤的水分流失相当快，要特别注重补水哦！我们新出了一款产品，使用后皮肤水汪汪的，滋润而不油腻。是我们卖的最好的明星产品之一。


用户问题：瑶瑶和小盖是什么关系



In [209]:
ChatCompletion("", user_prompt)["choices"][0]["message"]["content"]

'瑶瑶和小盖是墨问西东团队中的成员，他们是同事关系。瑶瑶是墨问西东的颜值担当，而小盖则是墨问西东团队中最帅的男人。'

### 非常好，那就开始工程化集成

In [ ]:
# 上代码